### Importování dat

In [2]:
import pandas as pd

df = pd.read_csv('psp_records_list.csv')
df.tail()

,Line
19816,"pan,poslanec,mikuláš,ferjenčík,faktickou,pozná..."
19817,"já,se,také,pokusím,vystoupit,naposledy,každopá..."
19818,"pan,předseda,kalousek"
19819,"rámci,konsenzuální,diskuse,navrhuji,kompromis,..."
19820,"tuto,chvíli,nemám,nikoho,přihlášeného,do,obecn..."


### Zbavení se diakritky a změnění písmen na malé

In [3]:
def lower_text(df):
    for index in range(df.size):
        df.values[index][0] = df.values[index][0].lower()
        
df_lower = lower_text(df)

In [4]:
import unicodedata

def deaccent(unistr):
    return "".join(aChar 
                   for aChar in unicodedata.normalize("NFD", unistr) 
                   if not unicodedata.combining(aChar))

for i in range(df.size):
    df.iloc[i,0] = deaccent(df.iloc[i,0])

### Listy vět sestavené z jednotlivých slov

In [5]:
def get_sentences(df):
    df_values = df.values.tolist()
    all_sentences = []
    for index in range(df.size):
        all_sentences.append(df_values[index][0].split(',')) 
    return all_sentences

all_sentences = get_sentences(df)

### Nalezení a zbavení se nejfrekventovanějších slov, nemající velký význam

In [25]:
from nltk.probability import FreqDist

fdist = FreqDist()
for i in range(df.size):
    for j in range(len(all_sentences[i])):
        fdist[all_sentences[i][j]] += 1

In [29]:
most_common = []
for i in range(41):
    most_common.append(fdist.most_common()[i][0])

In [34]:
# adding more stopwords in most_common list

words_to_add = ['tedy', 'bych', 'tomu', 'kdo', 
                'toho', 'tam', 'byl', 'jeste', 'proti',
                'kdyz', 'ta', 'tim', 'my', 'pokud', 'ktera',
                'take', 'bylo', 'od', 'tech', 'ma', 'jenom',
                'byt', 'mate', 'kteri', 'tento', 'tu',
                'budeme', 'ted', 'po', 'vam',
                'jestli', 'vas', 'byla', 'kdy', 'te',
                'podle', 'mame', 'nas', 'abychom', 'teto',
                'jejich']

most_common += words_to_add

In [32]:
def remove_common_words(all_sentences):
    for word in most_common:
        for index in range(df.size):
            while word in all_sentences[index]:
                all_sentences[index].remove(word)
                
remove_common_words(all_sentences)

# Word2Vec - knihovna gensim

In [35]:
from gensim.models import Word2Vec

word2vec = Word2Vec(all_sentences, size=200, min_count=5, window=10)

In [36]:
vocabulary = word2vec.wv.vocab

### Slovo reprezentováno jako vektor

In [37]:
word2vec.wv['zeman']

array([-0.00598075,  0.11503159, -0.06782123, -0.11009414,  0.00785042,
        0.07232863,  0.00392036, -0.04616841,  0.10157858, -0.14617252,
        0.02668748,  0.04319647, -0.15760352,  0.20529316,  0.02238465,
       -0.11309972,  0.10021774, -0.02503912,  0.00708629,  0.15442766,
       -0.02742318, -0.21829085, -0.05150516, -0.15630211,  0.0366746 ,
        0.18000321,  0.03010106,  0.11410537,  0.04421768,  0.2519884 ,
        0.08120463,  0.02799019,  0.13207571,  0.21930207, -0.03714715,
        0.10842385,  0.16728865, -0.15013503,  0.01116903, -0.05705654,
        0.03296358,  0.14358777,  0.2152675 , -0.08954191,  0.02832232,
        0.08039251, -0.06847469,  0.03791984, -0.39408553,  0.28117216,
       -0.02397783,  0.02365965, -0.10647084, -0.15061706, -0.01724453,
       -0.03471172, -0.24066004,  0.15469134, -0.02351044,  0.11742493,
       -0.02309477, -0.27808437,  0.15185785,  0.17496651,  0.21515678,
       -0.06565751, -0.00236587,  0.00980185,  0.1406214 ,  0.16

### Nalezení n nejpodobnějších slov

In [38]:
search_word = input("> ")
word2vec.wv.most_similar(deaccent(search_word).lower())

> Miloš


[('zeman', 0.9147958755493164),
 ('byval', 0.8698874711990356),
 ('dar', 0.8605464696884155),
 ('nasel', 0.8512917160987854),
 ('bojovnici', 0.8479036092758179),
 ('argumentujete', 0.8443666696548462),
 ('palestincum', 0.8420073986053467),
 ('nechapete', 0.8375751972198486),
 ('mily', 0.8353646397590637),
 ('nelhal', 0.832680344581604)]

In [39]:
search_word = input("> ")
word2vec.wv.most_similar(deaccent(search_word).lower())

> Zeman


[('milos', 0.9147959351539612),
 ('sef', 0.8843529224395752),
 ('wolf', 0.8726611137390137),
 ('sobotka', 0.8434608578681946),
 ('primator', 0.8425475358963013),
 ('miroslav', 0.8395073413848877),
 ('vytkl', 0.8383297324180603),
 ('doktor', 0.8375393748283386),
 ('dar', 0.8364630341529846),
 ('postavil', 0.8352385759353638)]

In [40]:
search_word = input("> ")
word2vec.wv.most_similar(deaccent(search_word).lower())

> TOP


[('piratu', 0.9511512517929077),
 ('ods', 0.9431631565093994),
 ('stan', 0.9276646971702576),
 ('kscm', 0.922848641872406),
 ('poslaneckeho', 0.908523678779602),
 ('klubu', 0.8991142511367798),
 ('klub', 0.8968909978866577),
 ('jmenem', 0.8785552978515625),
 ('predsedum', 0.8762214183807373),
 ('lidovcu', 0.8696448802947998)]

In [41]:
search_word = input("> ")
word2vec.wv.most_similar(deaccent(search_word).lower())

> ANO


[('hnuti', 0.8012479543685913),
 ('cssd', 0.7893269062042236),
 ('jednotni', 0.7253208756446838),
 ('kscm', 0.696901798248291),
 ('pravda', 0.6930057406425476),
 ('komunistu', 0.6881325840950012),
 ('asocialni', 0.6860584020614624),
 ('ods', 0.6701514720916748),
 ('lavic', 0.6480659246444702),
 ('populista', 0.6322942972183228)]